<a href="https://colab.research.google.com/github/Zilleplus/MachineLearning/blob/feature%2Fadd-pytorch-examples/AutoDiffTorchExamples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True) # set to false if you want it as parameter.
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

A function that we apply to tensors to construct computational graph is in fact an object of class Function. **This object knows how to compute the function in the forward direction, and also how to compute its derivative during the backward propagation step.** A reference to the backward propagation function is stored in grad_fn property of a tensor.

In [ ]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7fc038d50510>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fc038d50110>


In [ ]:
loss.backward()
print(w.grad)s
print(b.grad)

tensor([[0.3033, 0.2325, 0.0358],
        [0.3033, 0.2325, 0.0358],
        [0.3033, 0.2325, 0.0358],
        [0.3033, 0.2325, 0.0358],
        [0.3033, 0.2325, 0.0358]])
None


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1013: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at  aten/src/ATen/core/TensorBody.h:417.)
  return self._grad


In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

# evaluate the same Z but without gradient tracking:
with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [ ]:
z = torch.matmul(x, w)+b
z_det = z.detach() # alternative to .no_grad()
print(z_det.requires_grad)

False


Pytorch also allows you to calculate the jacobian product $v^T J$. The Size of $v$ should be the same as the size of the original tensor, with respoct to which we want to compute the product.

In [ ]:
inp = torch.eye(5, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])

Second call
 tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.],
        [4., 4., 4., 4., 8.]])

Call after zeroing gradients
 tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.],
        [2., 2., 2., 2., 4.]])
